# 🗄️ Vector Database Comparison per K-RSS

Questo notebook confronta le principali opzioni di vector database per memorizzare gli embeddings dei video.

## Opzioni considerate

| Database | Tipo | Pro | Contro |
|----------|------|-----|--------|
| **ChromaDB** | Embedded | Semplice, persistenza, metadata filtering | Meno performante su grandi scale |
| **FAISS** | Library | Velocissimo, Facebook-backed | Solo indice, no metadata nativo |
| **Qdrant** | Server/Embedded | Moderno, filtering avanzato | Più complesso da configurare |
| **LanceDB** | Embedded | Serverless, columnar | Relativamente nuovo |

Per un progetto accademico con ~1000-10000 video, **ChromaDB** o **FAISS** sono le scelte migliori.

## 1. Setup e Caricamento Dati

In [8]:
# Installazione dipendenze (eseguire solo la prima volta)
!pip install chromadb faiss-cpu sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 17.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 20.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 18.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 15.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 17.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 229.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import json
import time
import numpy as np
from pathlib import Path
from dataclasses import dataclass
from typing import List, Dict, Optional

# Percorso dati (adatta se esegui localmente vs Docker)
DATA_PATH = Path("../data/raw/scraped_videos.json")
if not DATA_PATH.exists():
    DATA_PATH = Path("/app/data/raw/scraped_videos.json")  # Docker path

print(f"Data path: {DATA_PATH}")
print(f"File exists: {DATA_PATH.exists()}")

📁 Data path: ../data/raw/scraped_videos.json
✅ File exists: True


In [ ]:
# Carica i video scraped
with open(DATA_PATH, 'r') as f:
    data = json.load(f)

videos = data.get('videos', [])
print(f"Video caricati: {len(videos)}")

# Mostra un esempio
if videos:
    print("\n🎬 Esempio video:")
    print(json.dumps(videos[0], indent=2, ensure_ascii=False)[:500])

📺 Video caricati: 135

🎬 Esempio video:
{
  "video_id": "FE-hM1kRK4Y",
  "channel_id": "UCYO_jab_esuFRV4b17AJtAw",
  "channel_name": "3Blue1Brown",
  "title": "Why Laplace transforms are so useful",
  "description": "Studying the forced harmonic oscillator by taking a Laplace transform and studying its poles.\nInstead of sponsored ad reads, these lessons are funded directly by viewers: https://3b1b.co/support\nAn equally valuable form of support is to simply share the videos.\nHome page: https://www.3blue1brown.com\n\nChapter on the L


## 2. Generazione Embeddings con Sentence Transformers

In [ ]:
from sentence_transformers import SentenceTransformer

# Modelli consigliati:
# - 'all-MiniLM-L6-v2': veloce, 384 dim (buono per prototipazione)
# - 'all-mpnet-base-v2': più accurato, 768 dim
# - 'paraphrase-multilingual-MiniLM-L12-v2': multilingua

MODEL_NAME = 'all-MiniLM-L6-v2'
print(f"Caricamento modello: {MODEL_NAME}")

model = SentenceTransformer(MODEL_NAME)
print(f"Modello caricato! Dimensione embedding: {model.get_sentence_embedding_dimension()}")

🤖 Caricamento modello: all-MiniLM-L6-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Modello caricato! Dimensione embedding: 384


In [ ]:
# Prepara i testi per l'embedding
# Combiniamo titolo + descrizione per contesto più ricco

def prepare_text(video: dict) -> str:
    """Combina titolo e descrizione per l'embedding."""
    title = video.get('title', '')
    description = video.get('description', '')[:500]  # Tronca descrizioni lunghe
    return f"{title}. {description}".strip()

texts = [prepare_text(v) for v in videos]
video_ids = [v.get('video_id', f'video_{i}') for i, v in enumerate(videos)]

print(f"Testi preparati: {len(texts)}")
print(f"\nEsempio testo:\n{texts[0][:200]}...")

📝 Testi preparati: 135

📄 Esempio testo:
Why Laplace transforms are so useful. Studying the forced harmonic oscillator by taking a Laplace transform and studying its poles.
Instead of sponsored ad reads, these lessons are funded directly by ...


In [ ]:
# Genera embeddings
print("Generazione embeddings...")
start_time = time.time()

embeddings = model.encode(texts, show_progress_bar=True, convert_to_numpy=True)

elapsed = time.time() - start_time
print(f"\nEmbeddings generati in {elapsed:.2f}s")
print(f"Shape: {embeddings.shape}")
print(f"Velocità: {len(texts)/elapsed:.1f} video/sec")

⏳ Generazione embeddings...


Batches:   0%|          | 0/5 [00:00<?, ?it/s]


✅ Embeddings generati in 3.48s
📊 Shape: (135, 384)
⚡ Velocità: 38.8 video/sec


## 3. Opzione A: ChromaDB

**ChromaDB** è un vector database embedded, perfetto per progetti locali.

### Vantaggi:
- ✅ API semplice e intuitiva
- ✅ Persistenza su disco integrata
- ✅ Metadata filtering (filtra per canale, categoria, etc.)
- ✅ Supporto nativo per sentence-transformers

### Svantaggi:
- ❌ Meno performante di FAISS per ricerche massive
- ❌ Non adatto per milioni di vettori

In [ ]:
import chromadb
from chromadb.config import Settings

# Inizializza ChromaDB (persistente su disco)
CHROMA_PATH = Path("../data/embeddings/chroma_db")
CHROMA_PATH.mkdir(parents=True, exist_ok=True)

chroma_client = chromadb.PersistentClient(path=str(CHROMA_PATH))
print(f"ChromaDB inizializzato in: {CHROMA_PATH}")

✅ ChromaDB inizializzato in: ../data/embeddings/chroma_db


In [ ]:
# Crea o ottieni la collection
collection = chroma_client.get_or_create_collection(
    name="krss_videos",
    metadata={"hnsw:space": "cosine"}  # Usa similarità coseno
)

print(f"Collection: {collection.name}")
print(f"Documenti esistenti: {collection.count()}")

📚 Collection: krss_videos
📊 Documenti esistenti: 0


In [ ]:
# Inserisci i video (con metadata!)
# ChromaDB permette di salvare metadata per ogni documento

if collection.count() == 0:
    print("Inserimento video in ChromaDB...")
    start_time = time.time()
    
    # Prepara metadata per ogni video
    metadatas = []
    for v in videos:
        metadatas.append({
            "channel_id": v.get("channel_id", ""),
            "channel_name": v.get("channel_name", ""),
            "category": v.get("category_name", "Unknown"),
            "published_at": v.get("published_at", ""),
        })
    
    # Batch insert
    collection.add(
        ids=video_ids,
        embeddings=embeddings.tolist(),
        documents=texts,
        metadatas=metadatas
    )
    
    elapsed = time.time() - start_time
    print(f"Inseriti {len(videos)} video in {elapsed:.2f}s")
else:
    print(f"Collection già popolata con {collection.count()} documenti")

⏳ Inserimento video in ChromaDB...
✅ Inseriti 135 video in 0.12s


In [ ]:
# Test: Ricerca per similarità
query = "machine learning tutorial for beginners"
query_embedding = model.encode([query])[0]

print(f"Query: '{query}'\n")

start_time = time.time()
results = collection.query(
    query_embeddings=[query_embedding.tolist()],
    n_results=5,
    include=["documents", "metadatas", "distances"]
)
search_time = (time.time() - start_time) * 1000

print(f"Tempo ricerca: {search_time:.2f}ms\n")
print("Risultati:")
for i, (doc, meta, dist) in enumerate(zip(
    results['documents'][0], 
    results['metadatas'][0], 
    results['distances'][0]
)):
    similarity = 1 - dist  # Converti distanza in similarità
    print(f"\n{i+1}. [{similarity:.3f}] {doc[:80]}...")
    print(f"\tCanale: {meta.get('channel_name', 'N/A')} | Categoria: {meta.get('category', 'N/A')}")

🔍 Query: 'machine learning tutorial for beginners'

⚡ Tempo ricerca: 3.05ms

📺 Risultati:

1. [0.413] Unitree G1 LiDAR, SLAM, navigation and control. Dev w/ G1 Humanoid P.2. Doing SL...
   📺 Canale: sentdex | Categoria: Education

2. [0.407] Programming with LLM Agents in 2025. Some tips and tricks for using modern LLM a...
   📺 Canale: sentdex | Categoria: Education

3. [0.407] Visualizing Neural Network Internals. Visualizing some of the internals of a neu...
   📺 Canale: sentdex | Categoria: Education

4. [0.404] Unboxing the Unitree G1 Edu Humanoid. Initial experience with unboxing, setting ...
   📺 Canale: sentdex | Categoria: Education

5. [0.391] DeepMind’s New Game AI Just Made History. ❤️ Check out Lambda here and sign up f...
   📺 Canale: Two Minute Papers | Categoria: Science & Technology


In [ ]:
# Feature speciale ChromaDB: Filtering per metadata!
# Cerca solo in una categoria specifica

results_filtered = collection.query(
    query_embeddings=[query_embedding.tolist()],
    n_results=3,
    where={"category": "Education"},  # Filtra per categoria
    include=["documents", "metadatas"]
)

print("Risultati filtrati per categoria 'Education':")
for doc, meta in zip(results_filtered['documents'][0], results_filtered['metadatas'][0]):
    print(f"  • {doc[:60]}... [{meta.get('category')}]")

🎯 Risultati filtrati per categoria 'Education':
  • Unitree G1 LiDAR, SLAM, navigation and control. Dev w/ G1 Hu... [Education]
  • Programming with LLM Agents in 2025. Some tips and tricks fo... [Education]
  • Visualizing Neural Network Internals. Visualizing some of th... [Education]


## 4. Opzione B: FAISS (Facebook AI Similarity Search)

**FAISS** è una libreria ottimizzata per ricerca di similarità ad alte prestazioni.

### Vantaggi:
- ✅ Estremamente veloce (anche con milioni di vettori)
- ✅ Supporto GPU
- ✅ Diversi tipi di indice (Flat, IVF, HNSW)

### Svantaggi:
- ❌ Solo indice vettoriale (no metadata nativi)
- ❌ Devi gestire la mappatura ID → metadata separatamente
- ❌ API meno intuitiva

In [ ]:
import faiss

# FAISS richiede float32
embeddings_f32 = embeddings.astype('float32')

# Normalizza per cosine similarity
faiss.normalize_L2(embeddings_f32)

# Crea indice (Flat = brute force, più accurato)
dimension = embeddings_f32.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product dopo normalizzazione = cosine

print(f"Dimensione vettori: {dimension}")
print(f"Indice FAISS creato: {type(index).__name__}")

📊 Dimensione vettori: 384
✅ Indice FAISS creato: IndexFlatIP


In [ ]:
# Aggiungi vettori all'indice
start_time = time.time()
index.add(embeddings_f32)
elapsed = time.time() - start_time

print(f"Aggiunti {index.ntotal} vettori in {elapsed*1000:.2f}ms")

✅ Aggiunti 135 vettori in 0.93ms


In [ ]:
# Test: Ricerca per similarità
query = "machine learning tutorial for beginners"
query_embedding = model.encode([query]).astype('float32')
faiss.normalize_L2(query_embedding)

print(f"Query: '{query}'\n")

k = 5  # Top-K risultati
start_time = time.time()
scores, indices = index.search(query_embedding, k)
search_time = (time.time() - start_time) * 1000

print(f"⚡ Tempo ricerca FAISS: {search_time:.4f}ms\n")
print("📺 Risultati:")
for i, (idx, score) in enumerate(zip(indices[0], scores[0])):
    video = videos[idx]
    print(f"\n{i+1}. [{score:.3f}] {video.get('title', 'N/A')[:70]}...")
    print(f"\tCanale: {video.get('channel_name', 'N/A')}")

🔍 Query: 'machine learning tutorial for beginners'

⚡ Tempo ricerca FAISS: 0.1686ms

📺 Risultati:

1. [0.413] Unitree G1 LiDAR, SLAM, navigation and control. Dev w/ G1 Humanoid P.2...
   📺 Canale: sentdex

2. [0.407] Programming with LLM Agents in 2025...
   📺 Canale: sentdex

3. [0.407] Visualizing Neural Network Internals...
   📺 Canale: sentdex

4. [0.404] Unboxing the Unitree G1 Edu Humanoid...
   📺 Canale: sentdex

5. [0.391] DeepMind’s New Game AI Just Made History...
   📺 Canale: Two Minute Papers


In [ ]:
# Salva/Carica indice FAISS
FAISS_PATH = Path("../data/embeddings/faiss_index")
FAISS_PATH.mkdir(parents=True, exist_ok=True)

# Salva
faiss.write_index(index, str(FAISS_PATH / "videos.index"))
print(f"Indice salvato in: {FAISS_PATH / 'videos.index'}")

# Per FAISS serve salvare anche la mappatura ID separatamente!
import pickle
with open(FAISS_PATH / "id_mapping.pkl", "wb") as f:
    pickle.dump({"video_ids": video_ids, "videos": videos}, f)
print(f"Mappatura salvata in: {FAISS_PATH / 'id_mapping.pkl'}")

💾 Indice salvato in: ../data/embeddings/faiss_index/videos.index
💾 Mappatura salvata in: ../data/embeddings/faiss_index/id_mapping.pkl


## 5. Benchmark: ChromaDB vs FAISS

In [ ]:
import statistics

# Test queries
test_queries = [
    "python programming tutorial",
    "how to cook pasta",
    "artificial intelligence explained",
    "gaming walkthrough",
    "music production tips"
]

n_runs = 10  # Ripetizioni per media

chroma_times = []
faiss_times = []

for query in test_queries:
    query_emb = model.encode([query])
    query_emb_f32 = query_emb.astype('float32')
    faiss.normalize_L2(query_emb_f32)
    
    # ChromaDB
    for _ in range(n_runs):
        start = time.time()
        _ = collection.query(query_embeddings=[query_emb[0].tolist()], n_results=10)
        chroma_times.append((time.time() - start) * 1000)
    
    # FAISS
    for _ in range(n_runs):
        start = time.time()
        _ = index.search(query_emb_f32, 10)
        faiss_times.append((time.time() - start) * 1000)

print("📊 BENCHMARK RISULTATI")
print("=" * 40)
print(f"\nChromaDB:")
print(f"\tMedia: {statistics.mean(chroma_times):.3f}ms")
print(f"\tMediana: {statistics.median(chroma_times):.3f}ms")
print(f"\tMin/Max: {min(chroma_times):.3f}ms / {max(chroma_times):.3f}ms")

print(f"\nFAISS:")
print(f"\tMedia: {statistics.mean(faiss_times):.3f}ms")
print(f"\tMediana: {statistics.median(faiss_times):.3f}ms")
print(f"\tMin/Max: {min(faiss_times):.3f}ms / {max(faiss_times):.3f}ms")

speedup = statistics.mean(chroma_times) / statistics.mean(faiss_times)
print(f"\nFAISS è {speedup:.1f}x più veloce di ChromaDB")

📊 BENCHMARK RISULTATI

🟠 ChromaDB:
   Media: 1.883ms
   Mediana: 1.787ms
   Min/Max: 1.401ms / 3.164ms

🔵 FAISS:
   Media: 0.023ms
   Mediana: 0.018ms
   Min/Max: 0.016ms / 0.085ms

⚡ FAISS è 82.8x più veloce di ChromaDB


## 6. Raccomandazione Finale

### Per K-RSS utiizzeremo: **ChromaDB** 

**Motivazioni:**

1. **Metadata Filtering**: Fondamentale per l'obiettivo del nostro progetto
   - Filtrare per categoria (Education, Gaming, etc.)
   - Filtrare per canale

2. **Persistenza Integrata**: Salva automaticamente su disco

3. **Scala del Progetto**: Con ~10K video, la differenza di performance è trascurabile

4. **Semplicità**: API più pulita, meno codice da mantenere

5. **Documenti + Embeddings**: Possiamo salvare il testo originale insieme al vettore


In [ ]:
# Riepilogo configurazione consigliata per K-RSS

RECOMMENDED_CONFIG = {
    "vector_db": "ChromaDB",
    "embedding_model": "all-MiniLM-L6-v2",  # Veloce, 384 dim
    "embedding_dimension": 384,
    "similarity_metric": "cosine",
    "persistence_path": "data/embeddings/chroma_db",
    "collection_name": "krss_videos",
    "metadata_fields": [
        "channel_id",
        "channel_name", 
        "category",
        "published_at"
    ]
}

print("Configurazione Consigliata per K-RSS:")
print(json.dumps(RECOMMENDED_CONFIG, indent=2))

📋 Configurazione Consigliata per K-RSS:
{
  "vector_db": "ChromaDB",
  "embedding_model": "all-MiniLM-L6-v2",
  "embedding_dimension": 384,
  "similarity_metric": "cosine",
  "persistence_path": "data/embeddings/chroma_db",
  "collection_name": "krss_videos",
  "metadata_fields": [
    "channel_id",
    "channel_name",
    "category",
    "published_at"
  ]
}
